# Explorando banco de dados ENEM 2016

Existem muitos valores ausentes, tanto no conjunto de treino quanto no de deste, além de que existem menos colunas no teste do que no treino.

OBJETIVO: prever a nota de matemática (não existe no treino)

In [ ]:
library(tidyverse)
library(readxl)

In [4]:
treino <- read_csv('train.csv')
teste <- read_csv('test.csv')

Warning message:
"Missing column names filled in: 'X1' [1]"
Parsed with column specification:
cols(
  .default = col_double(),
  NU_INSCRICAO = col_character(),
  NO_MUNICIPIO_RESIDENCIA = col_character(),
  SG_UF_RESIDENCIA = col_character(),
  TP_SEXO = col_character(),
  NO_MUNICIPIO_NASCIMENTO = col_character(),
  SG_UF_NASCIMENTO = col_character(),
  NO_MUNICIPIO_ESC = col_character(),
  SG_UF_ESC = col_character(),
  NO_ENTIDADE_CERTIFICACAO = col_character(),
  SG_UF_ENTIDADE_CERTIFICACAO = col_character(),
  NO_MUNICIPIO_PROVA = col_character(),
  SG_UF_PROVA = col_character(),
  CO_PROVA_CN = col_character(),
  CO_PROVA_CH = col_character(),
  CO_PROVA_LC = col_character(),
  CO_PROVA_MT = col_character(),
  TX_RESPOSTAS_CN = col_character(),
  TX_RESPOSTAS_CH = col_character(),
  TX_RESPOSTAS_LC = col_character(),
  TX_RESPOSTAS_MT = col_character()
  # ... with 40 more columns
)

See spec(...) for full column specifications.

Parsed with column specification:
cols(
  .defaul

In [3]:
glimpse(treino)

Observations: 13,730
Variables: 167
$ X1                          <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...
$ NU_INSCRICAO                <chr> "ed50e8aaa58e7a806c337585efee9ca41f1eb1...
$ NU_ANO                      <dbl> 2016, 2016, 2016, 2016, 2016, 2016, 201...
$ CO_MUNICIPIO_RESIDENCIA     <dbl> 4314902, 2304707, 2304400, 3304557, 130...
$ NO_MUNICIPIO_RESIDENCIA     <chr> "Porto Alegre", "Granja", "Fortaleza", ...
$ CO_UF_RESIDENCIA            <dbl> 43, 23, 23, 33, 13, 29, 51, 43, 26, 31,...
$ SG_UF_RESIDENCIA            <chr> "RS", "CE", "CE", "RJ", "AM", "BA", "MT...
$ NU_IDADE                    <dbl> 24, 17, 21, 25, 28, 18, 18, 18, 16, 24,...
$ TP_SEXO                     <chr> "M", "F", "F", "F", "M", "F", "F", "F",...
$ TP_ESTADO_CIVIL             <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, NA,...
$ TP_COR_RACA                 <dbl> 1, 3, 3, 0, 2, 1, 3, 1, 3, 4, 3, 3, 3, ...
$ TP_NACIONALIDADE            <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ...
$ CO_MUNICIPIO_N

In [5]:
glimpse(teste)

Observations: 4,576
Variables: 47
$ NU_INSCRICAO           <chr> "73ff9fcc02f0a99919906c942c2e1a1042cdcf98", ...
$ CO_UF_RESIDENCIA       <dbl> 41, 21, 23, 15, 41, 43, 35, 23, 51, 15, 24, ...
$ SG_UF_RESIDENCIA       <chr> "PR", "MA", "CE", "PA", "PR", "RS", "SP", "C...
$ NU_IDADE               <dbl> 22, 26, 21, 27, 18, 18, 17, 17, 18, 17, 20, ...
$ TP_SEXO                <chr> "F", "F", "M", "F", "M", "M", "F", "F", "F",...
$ TP_COR_RACA            <dbl> 3, 3, 1, 3, 1, 3, 1, 3, 3, 3, 3, 1, 3, 2, 1,...
$ TP_NACIONALIDADE       <dbl> 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 4,...
$ TP_ST_CONCLUSAO        <dbl> 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1,...
$ TP_ANO_CONCLUIU        <dbl> 5, 8, 0, 8, 0, 1, 0, 0, 0, 0, 2, 10, 5, 1, 2...
$ TP_ESCOLA              <dbl> 1, 1, 2, 1, 2, 1, 3, 2, 2, 2, 1, 1, 1, 1, 1,...
$ TP_ENSINO              <dbl> NA, NA, 3, NA, 1, NA, 1, 1, 1, 1, NA, NA, NA...
$ IN_TREINEIRO           <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ TP_DEPENDENCIA_A

### Colunas a serem removidas

Existem colunas que são repetidas, exemplo: sigla do estado e código do estado. 

Por padrão será mantida apenas o código.

O **ID** do candidato (**NU_INCRICAO**) também deve ser removido devido apresentar alta cardinalidade já que cada registro apresenta um identificador único.

O código da escola também será removido devido alta cardinalidade e por essa informação estar presente na cidade/estado.

In [24]:
colsRemov = c(
    # removendo ID do candidato (alta cardinalidade)
    'NU_INSCRICAO',
    'NU_ANO',
    # tiram ID de nome, deixando os CODIGO
    'NO_MUNICIPIO_RESIDENCIA',
    # esse preciso deixar pois existe conjunto TESTE
    #'SG_UF_RESIDENCIA',
    # então retiro o código
    'CO_UF_RESIDENCIA',
    'NO_MUNICIPIO_NASCIMENTO',
    'SG_UF_NASCIMENTO',
    # informação de localização já existe no municipio
    'CO_ESCOLA',
    'CO_MUNICIPIO_ESC',
    'NO_MUNICIPIO_ESC',
    'CO_UF_ESC',
    'SG_UF_ESC',
    # aparentemente informação irrelevante
    'NO_ENTIDADE_CERTIFICACAO',
    'NO_MUNICIPIO_PROVA',
    'SG_UF_PROVA')

In [26]:
# colunas do teste
colsTeste = names(teste)

encontrouLogic = map_lgl(
    colsTeste,
    function(x) str_detect(paste(colsRemov, collapse = ''), x)
)
                         
print(str_c('Colunas que serão removidas e se apresentam no teste:  ',colsTeste[encontrouLogic]))

[1] "Colunas que serão removidas e se apresentam no teste:  NU_INSCRICAO"


In [49]:
# removendo as colunas 
treino <- treino %>% 
    select(-colsRemov, -X1)

# Tipos das colunas

4 tipos de colunas presente nos dados:
* numérica discreta - número de irmãos
* numérica contínua - peso, tempo
* categórica nominal - não existe ordem de relevância
* categórica ordinal - existe ordem

In [73]:
numDisc = c()
numCont = c()
# binárias vão entrar em nominal
categNom = c()
categOrd = c()

In [74]:
# classificando as colunas de acordo com o tipo delas
for(col in names(treino)){
    
    if(str_detect(col, 'Q0')){
        if(str_detect(col, 'Q018|Q020|Q021|Q023|Q025')){
            categNom = c(categNom, col)
        }else{
            categOrd = c(categOrd, col)
        }
    }
    
    if(str_detect(col, 'IN_') | str_detect(col, 'CO_|SG_|TP_')){
        categNom = c(categNom, col)
    }
    
    if(str_detect(col, 'NU_IDADE')){
        numDisc = c(numDisc, col)
    }
    
    if(str_detect(col, 'NU_NOTA_')){
        if(str_detect(col , 'COMP|REDACAO')){
            numDisc = c(numDisc, col)
        }else{
            numCont = c(numCont, col)
        }
        
    }
    
    if(str_detect(col, 'TX_')){
        categNom = c(categNom, col)
    }
    
}

In [80]:
categNom

[1] "CO_MUNICIPIO_RESIDENCIA"     "CO_UF_RESIDENCIA"           
 [3] "SG_UF_RESIDENCIA"            "TP_SEXO"                    
 [5] "TP_ESTADO_CIVIL"             "TP_COR_RACA"                
 [7] "TP_NACIONALIDADE"            "CO_MUNICIPIO_NASCIMENTO"    
 [9] "CO_UF_NASCIMENTO"            "TP_ST_CONCLUSAO"            
[11] "TP_ANO_CONCLUIU"             "TP_ESCOLA"                  
[13] "TP_ENSINO"                   "IN_TREINEIRO"               
[15] "TP_DEPENDENCIA_ADM_ESC"      "TP_LOCALIZACAO_ESC"         
[17] "TP_SIT_FUNC_ESC"             "IN_BAIXA_VISAO"             
[19] "IN_CEGUEIRA"                 "IN_SURDEZ"                  
[21] "IN_DEFICIENCIA_AUDITIVA"     "IN_SURDO_CEGUEIRA"          
[23] "IN_DEFICIENCIA_FISICA"       "IN_DEFICIENCIA_MENTAL"      
[25] "IN_DEFICIT_ATENCAO"          "IN_DISLEXIA"                
[27] "IN_DISCALCULIA"              "IN_AUTISMO"                 
[29] "IN_VISAO_MONOCULAR"          "IN_OUTRA_DEF"               
[31] "IN_SABATISTA"                "IN_GESTANTE"                
[33] "IN_LACTANTE"                 "IN_IDOSO"                   
[35] "IN_ESTUDA_CLASSE_HOSPITALAR" "IN_SEM_RECURSO"             
[37] "IN_BRAILLE"                  "IN_AMPLIADA_24"             
[39] "IN_AMPLIADA_18"              "IN_LEDOR"                   
[41] "IN_ACESSO"                   "IN_TRANSCRICAO"             
[43] "IN_LIBRAS"                   "IN_LEITURA_LABIAL"          
[45] "IN_MESA_CADEIRA_RODAS"       "IN_MESA_CADEIRA_SEPARADA"   
[47] "IN_APOIO_PERNA"              "IN_GUIA_INTERPRETE"         
[49] "IN_MACA"                     "IN_COMPUTADOR"              
[51] "IN_CADEIRA_ESPECIAL"         "IN_CADEIRA_CANHOTO"         
[53] "IN_CADEIRA_ACOLCHOADA"       "IN_PROVA_DEITADO"           
[55] "IN_MOBILIARIO_OBESO"         "IN_LAMINA_OVERLAY"          
[57] "IN_PROTETOR_AURICULAR"       "IN_MEDIDOR_GLICOSE"         
[59] "IN_MAQUINA_BRAILE"           "IN_SOROBAN"                 
[61] "IN_MARCA_PASSO"              "IN_SONDA"                   
[63] "IN_MEDICAMENTOS"             "IN_SALA_INDIVIDUAL"         
[65] "IN_SALA_ESPECIAL"            "IN_SALA_ACOMPANHANTE"       
[67] "IN_MOBILIARIO_ESPECIFICO"    "IN_MATERIAL_ESPECIFICO"     
[69] "IN_NOME_SOCIAL"              "IN_CERTIFICADO"             
[71] "CO_UF_ENTIDADE_CERTIFICACAO" "SG_UF_ENTIDADE_CERTIFICACAO"
[73] "CO_MUNICIPIO_PROVA"          "CO_UF_PROVA"                
[75] "TP_PRESENCA_CN"              "TP_PRESENCA_CH"             
[77] "TP_PRESENCA_LC"              "TP_PRESENCA_MT"             
[79] "CO_PROVA_CN"                 "CO_PROVA_CH"                
[81] "CO_PROVA_LC"                 "CO_PROVA_MT"                
[83] "TX_RESPOSTAS_CN"             "TX_RESPOSTAS_CH"            
[85] "TX_RESPOSTAS_LC"             "TX_RESPOSTAS_MT"            
[87] "TP_LINGUA"                   "TX_GABARITO_CN"             
[89] "TX_GABARITO_CH"              "TX_GABARITO_LC"             
[91] "TX_GABARITO_MT"              "TP_STATUS_REDACAO"          
[93] "Q018"                        "Q020"                       
[95] "Q021"                        "Q023"                       
[97] "Q025"

In [95]:
 varsBin <- categNom[categNom %>%
                     map_lgl(
                         function(x) str_detect(x, 'IN_|TP_SEXO|TP_LINGUA')
                         )
                    ]

In [96]:
varsBin

[1] "TP_SEXO"                     "IN_TREINEIRO"               
 [3] "IN_BAIXA_VISAO"              "IN_CEGUEIRA"                
 [5] "IN_SURDEZ"                   "IN_DEFICIENCIA_AUDITIVA"    
 [7] "IN_SURDO_CEGUEIRA"           "IN_DEFICIENCIA_FISICA"      
 [9] "IN_DEFICIENCIA_MENTAL"       "IN_DEFICIT_ATENCAO"         
[11] "IN_DISLEXIA"                 "IN_DISCALCULIA"             
[13] "IN_AUTISMO"                  "IN_VISAO_MONOCULAR"         
[15] "IN_OUTRA_DEF"                "IN_SABATISTA"               
[17] "IN_GESTANTE"                 "IN_LACTANTE"                
[19] "IN_IDOSO"                    "IN_ESTUDA_CLASSE_HOSPITALAR"
[21] "IN_SEM_RECURSO"              "IN_BRAILLE"                 
[23] "IN_AMPLIADA_24"              "IN_AMPLIADA_18"             
[25] "IN_LEDOR"                    "IN_ACESSO"                  
[27] "IN_TRANSCRICAO"              "IN_LIBRAS"                  
[29] "IN_LEITURA_LABIAL"           "IN_MESA_CADEIRA_RODAS"      
[31] "IN_MESA_CADEIRA_SEPARADA"    "IN_APOIO_PERNA"             
[33] "IN_GUIA_INTERPRETE"          "IN_MACA"                    
[35] "IN_COMPUTADOR"               "IN_CADEIRA_ESPECIAL"        
[37] "IN_CADEIRA_CANHOTO"          "IN_CADEIRA_ACOLCHOADA"      
[39] "IN_PROVA_DEITADO"            "IN_MOBILIARIO_OBESO"        
[41] "IN_LAMINA_OVERLAY"           "IN_PROTETOR_AURICULAR"      
[43] "IN_MEDIDOR_GLICOSE"          "IN_MAQUINA_BRAILE"          
[45] "IN_SOROBAN"                  "IN_MARCA_PASSO"             
[47] "IN_SONDA"                    "IN_MEDICAMENTOS"            
[49] "IN_SALA_INDIVIDUAL"          "IN_SALA_ESPECIAL"           
[51] "IN_SALA_ACOMPANHANTE"        "IN_MOBILIARIO_ESPECIFICO"   
[53] "IN_MATERIAL_ESPECIFICO"      "IN_NOME_SOCIAL"             
[55] "IN_CERTIFICADO"              "TP_LINGUA"

In [112]:
treino %>% 
    select(categOrd) %>% 
    #select('Q018','Q020','Q021','Q023','Q025') %>% 
    gather(col, valor)  %>% 
    count(valor) 

valor,n
<chr>,<int>
0,9384
1,4843
10,52
11,22
12,8
13,8
14,6
15,3
16,1


In [110]:
letters

[1] "a" "b" "c" "d" "e" "f" "g" "h" "i" "j" "k" "l" "m" "n" "o" "p" "q" "r" "s"
[20] "t" "u" "v" "w" "x" "y" "z"

In [92]:
treino %>% 
    select(starts_with('Q0')) %>% 
    glimpse()

Observations: 13,730
Variables: 17
$ TP_SEXO                <chr> "M", "F", "F", "F", "M", "F", "F", "F", "F",...
$ TP_ESTADO_CIVIL        <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, NA, 0, 0...
$ TP_COR_RACA            <dbl> 1, 3, 3, 0, 2, 1, 3, 1, 3, 4, 3, 3, 3, 3, 3,...
$ TP_NACIONALIDADE       <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,...
$ TP_ST_CONCLUSAO        <dbl> 1, 2, 3, 1, 1, 1, 2, 1, 2, 4, 1, 2, 1, 1, 1,...
$ TP_ANO_CONCLUIU        <dbl> 4, 0, 0, 9, 4, 1, 0, 1, 0, 0, 8, 0, 7, 2, 2,...
$ TP_ESCOLA              <dbl> 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1,...
$ TP_ENSINO              <dbl> NA, 1, NA, NA, NA, NA, 1, NA, 1, NA, NA, 1, ...
$ TP_DEPENDENCIA_ADM_ESC <dbl> NA, 2, NA, NA, NA, NA, 2, NA, 2, NA, NA, 2, ...
$ TP_LOCALIZACAO_ESC     <dbl> NA, 1, NA, NA, NA, NA, 1, NA, 1, NA, NA, 1, ...
$ TP_SIT_FUNC_ESC        <dbl> NA, 1, NA, NA, NA, NA, 1, NA, 1, NA, NA, 1, ...
$ TP_PRESENCA_CN         <dbl> 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,...
$ TP_PRESENCA_CH 

In [47]:
categOrd

[1] "Q001" "Q002" "Q003" "Q004" "Q005" "Q006" "Q007" "Q008" "Q009" "Q010"
[11] "Q011" "Q012" "Q013" "Q014" "Q015" "Q016" "Q017" "Q019" "Q022" "Q024"
[21] "Q026" "Q027" "Q028" "Q029" "Q030" "Q031" "Q032" "Q033" "Q034" "Q035"
[31] "Q036" "Q037" "Q038" "Q039" "Q040" "Q041" "Q042" "Q043" "Q044" "Q045"
[41] "Q046" "Q047" "Q048" "Q049" "Q050"

In [82]:
numDisc

[1] "NU_IDADE"        "NU_NOTA_COMP1"   "NU_NOTA_COMP2"   "NU_NOTA_COMP3"  
[5] "NU_NOTA_COMP4"   "NU_NOTA_COMP5"   "NU_NOTA_REDACAO"

In [83]:
categNom

[1] "CO_MUNICIPIO_RESIDENCIA"     "CO_UF_RESIDENCIA"           
 [3] "SG_UF_RESIDENCIA"            "TP_SEXO"                    
 [5] "TP_ESTADO_CIVIL"             "TP_COR_RACA"                
 [7] "TP_NACIONALIDADE"            "CO_MUNICIPIO_NASCIMENTO"    
 [9] "CO_UF_NASCIMENTO"            "TP_ST_CONCLUSAO"            
[11] "TP_ANO_CONCLUIU"             "TP_ESCOLA"                  
[13] "TP_ENSINO"                   "IN_TREINEIRO"               
[15] "TP_DEPENDENCIA_ADM_ESC"      "TP_LOCALIZACAO_ESC"         
[17] "TP_SIT_FUNC_ESC"             "IN_BAIXA_VISAO"             
[19] "IN_CEGUEIRA"                 "IN_SURDEZ"                  
[21] "IN_DEFICIENCIA_AUDITIVA"     "IN_SURDO_CEGUEIRA"          
[23] "IN_DEFICIENCIA_FISICA"       "IN_DEFICIENCIA_MENTAL"      
[25] "IN_DEFICIT_ATENCAO"          "IN_DISLEXIA"                
[27] "IN_DISCALCULIA"              "IN_AUTISMO"                 
[29] "IN_VISAO_MONOCULAR"          "IN_OUTRA_DEF"               
[31] "IN_SABATISTA"                "IN_GESTANTE"                
[33] "IN_LACTANTE"                 "IN_IDOSO"                   
[35] "IN_ESTUDA_CLASSE_HOSPITALAR" "IN_SEM_RECURSO"             
[37] "IN_BRAILLE"                  "IN_AMPLIADA_24"             
[39] "IN_AMPLIADA_18"              "IN_LEDOR"                   
[41] "IN_ACESSO"                   "IN_TRANSCRICAO"             
[43] "IN_LIBRAS"                   "IN_LEITURA_LABIAL"          
[45] "IN_MESA_CADEIRA_RODAS"       "IN_MESA_CADEIRA_SEPARADA"   
[47] "IN_APOIO_PERNA"              "IN_GUIA_INTERPRETE"         
[49] "IN_MACA"                     "IN_COMPUTADOR"              
[51] "IN_CADEIRA_ESPECIAL"         "IN_CADEIRA_CANHOTO"         
[53] "IN_CADEIRA_ACOLCHOADA"       "IN_PROVA_DEITADO"           
[55] "IN_MOBILIARIO_OBESO"         "IN_LAMINA_OVERLAY"          
[57] "IN_PROTETOR_AURICULAR"       "IN_MEDIDOR_GLICOSE"         
[59] "IN_MAQUINA_BRAILE"           "IN_SOROBAN"                 
[61] "IN_MARCA_PASSO"              "IN_SONDA"                   
[63] "IN_MEDICAMENTOS"             "IN_SALA_INDIVIDUAL"         
[65] "IN_SALA_ESPECIAL"            "IN_SALA_ACOMPANHANTE"       
[67] "IN_MOBILIARIO_ESPECIFICO"    "IN_MATERIAL_ESPECIFICO"     
[69] "IN_NOME_SOCIAL"              "IN_CERTIFICADO"             
[71] "CO_UF_ENTIDADE_CERTIFICACAO" "SG_UF_ENTIDADE_CERTIFICACAO"
[73] "CO_MUNICIPIO_PROVA"          "CO_UF_PROVA"                
[75] "TP_PRESENCA_CN"              "TP_PRESENCA_CH"             
[77] "TP_PRESENCA_LC"              "TP_PRESENCA_MT"             
[79] "CO_PROVA_CN"                 "CO_PROVA_CH"                
[81] "CO_PROVA_LC"                 "CO_PROVA_MT"                
[83] "TX_RESPOSTAS_CN"             "TX_RESPOSTAS_CH"            
[85] "TX_RESPOSTAS_LC"             "TX_RESPOSTAS_MT"            
[87] "TP_LINGUA"                   "TX_GABARITO_CN"             
[89] "TX_GABARITO_CH"              "TX_GABARITO_LC"             
[91] "TX_GABARITO_MT"              "TP_STATUS_REDACAO"          
[93] "Q018"                        "Q020"                       
[95] "Q021"                        "Q023"                       
[97] "Q025"

In [ ]:
# separando binárias da nominais


In [61]:
notas <- treino %>% 
    select(starts_with('NU_'))

In [62]:
glimpse(notas)

Observations: 13,730
Variables: 11
$ NU_IDADE        <dbl> 24, 17, 21, 25, 28, 18, 18, 18, 16, 24, 29, 17, 27,...
$ NU_NOTA_CN      <dbl> 436.3, 474.5, NA, NA, NA, 439.7, 420.1, 619.6, 500....
$ NU_NOTA_CH      <dbl> 495.4, 544.1, NA, NA, NA, 583.2, 604.2, 625.8, 555....
$ NU_NOTA_LC      <dbl> 581.2, 599.0, NA, NA, NA, 410.9, 484.5, 611.2, 521....
$ NU_NOTA_MT      <dbl> 399.4, 459.8, NA, NA, NA, 364.5, 529.2, 566.7, 372....
$ NU_NOTA_COMP1   <dbl> 120, 140, NA, NA, NA, 120, 100, 140, 140, NA, 100, ...
$ NU_NOTA_COMP2   <dbl> 120, 120, NA, NA, NA, 120, 120, 120, 160, NA, 100, ...
$ NU_NOTA_COMP3   <dbl> 120, 120, NA, NA, NA, 120, 120, 120, 140, NA, 100, ...
$ NU_NOTA_COMP4   <dbl> 80, 120, NA, NA, NA, 160, 120, 120, 140, NA, 100, 1...
$ NU_NOTA_COMP5   <dbl> 80, 80, NA, NA, NA, 100, 100, 120, 100, NA, 120, 60...
$ NU_NOTA_REDACAO <dbl> 520, 580, NA, NA, NA, 620, 560, 620, 680, NA, 520, ...


In [67]:
notas %>% 
    select(NU_NOTA_COMP1:NU_NOTA_REDACAO) %>% 
    gather(col, valor) %>% count(col, valor) %>% .$valor
    ggplot(aes(col, valor))+
    geom_bar()+
    facet_wrap(~ col)
    

[1]    0   20   40   60   67   80  100  120  125  133  140  160  167  180  200
 [16]   NA    0   40   60   67   80  100  120  125  133  140  160  180  200   NA
 [31]    0   20   40   60   80  100  120  125  140  160  180  200   NA    0   20
 [46]   40   50   60   75   80  100  120  125  140  160  180  200   NA    0   20
 [61]   25   40   60   75   80  100  120  140  150  160  180  200   NA    0   40
 [76]   60   80  120  140  160  180  184  200  220  240  260  280  300  320  340
 [91]  360  380  400  420  440  460  466  480  500  520  540  560  566  580  600
[106]  620  640  650  660  680  700  720  740  760  780  800  820  840  860  880
[121]  900  920  940  960  980 1000   NA

ERROR: Error: `data` must be a data frame, or other object coercible by `fortify()`, not an S3 object with class uneval
Did you accidentally pass `aes()` to the `data` argument?


In [9]:
paste(colsRemov, collapse = '')

[1] "NU_INSCRICAONU_ANONO_MUNICIPIO_RESIDENCIASG_UF_RESIDENCIANO_MUNICIPIO_NASCIMENTOSG_UF_NASCIMENTOCO_ESCOLACO_MUNICIPIO_ESCNO_MUNICIPIO_ESCCO_UF_ESCSG_UF_ESCNO_ENTIDADE_CERTIFICACAONO_MUNICIPIO_PROVASG_UF_PROVA"

In [ ]:
summary(temp)

In [ ]:
df %>% 
    select(-(names(temp))) %>% 
    summary()

In [ ]:
df %>% 
    select(NU_NOTA_MT) %>% 
    mutate(nulo = is.na(NU_NOTA_MT)) %>% 
    .$nulo %>% 
    mean()

In [ ]:
names(df)

In [ ]:
glimpse(temp)

In [ ]:
temp %>% 
    ggplot(aes(CO_PROVA_CH)) +
    geom_bar()

In [ ]:
# assinalado e resposta correta (gabarito)
# deveria quebrar e trasnformar cada conjunto em uma coluna
# a quantidade de perguntas e respostas está OK
df %>% 
    select(starts_with('TX')) %>% 
    mutate_all(funs(nchar(.))) %>% 
    summary()

In [ ]:
temp %>% 
    select(!!colsRemov) %>% 
    glimpse()